데이터 처리하는 방법
- numpy 배열로 불러오면 됨
- 이미지 : Pillow, OpenCV
- 오디오 : SciPy, LibROSA
- 텍스트 : Python, Cython, NLTK, SpaCy

영상 분야에는 torchvision을 사용하면 좋음

### 이번 실습의 목표
CIFAR10 데이터를 사용하여 분류하는 것이 목표
- 분류 종류
> airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

### 이번 실습의 과정
1. torchvision을 사용하여 CIFAR10의 학습용 / 시험용 데이터셋을 불러오고, 정규화
2. CNN 모델 정의
3. 손실 함수 정의
4. 학습용 데이터를 사용하여 신경망 학습
5. 시험용 데이터를 사용하여 신경망 검사

### 1. CIFAR10 불러오기, and 정규화

In [1]:
# data load
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# torchvision 데이터 셋의 output은 [0,1]범위를 가지는 PILImage 이미지
# 이를 [-1,1] 범위로 정규화된 Tensor로 변환
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True,
                                        download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4,
                                          shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                        download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 4,
                                          shuffle = True, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# 학습용 이미지 랜덤 시각화
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
# 학습용 랜덤 이미지
dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))

print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 이미지를 보여주기 위한 함수

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# 학습용 이미지를 무작위로 가져오기
dataiter = iter(trainloader)
images, labels = dataiter.next()

# 이미지 보여주기
imshow(torchvision.utils.make_grid(images))
# 정답(label) 출력
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# 이미지를 보여주기 위한 함수

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [4]:
# 학습용 이미지를 무작위로 가져오기
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [5]:
images.size()

torch.Size([4, 3, 32, 32])

In [6]:
labels

tensor([7, 4, 4, 7])

In [7]:
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

horse  deer  deer horse


In [ ]:
imshow(torchvision.utils.make_grid(images))

In [8]:
torchvision.utils.make_grid(images)

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.1922,  ..., -1.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.1216,  ...,  0.0588,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.1765,  ..., -0.9686,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000, -0.0431,  ...,  0.0196,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0